In [1]:
import pytimber
ldb = pytimber.LoggingDB()

In [2]:
t1 = '2015-11-01 01:00:00.000'
t2 = '2015-12-01 00:59:00.000'

In [3]:
time = 'Timestamp (UTC_TIME)'
parameters = ['IP.NSRCGEN:BIASDISCAQNV', 'IP.NSRCGEN:GASSASAQN', 'IP.NSRCGEN:SOURCEHTAQNI', 'IP.SOLCEN.ACQUISITION:CURRENT', 'IP.SOLEXT.ACQUISITION:CURRENT', 'IP.NSRCGEN:OVEN1AQNP', 'IP.NSRCGEN:OVEN2AQNP']
measurments = ['ITF.BCT25:CURRENT']
columns_to_load = parameters + measurments

In [4]:
result = ldb.get(columns_to_load, t1, t2, unixtime=False)

In [5]:
import pandas as pd
df = pd.DataFrame(columns=[time])
df.set_index(time, inplace = True)

In [6]:
for column in columns_to_load:
    print("For column {} {} datapoints exist.".format(column, len(result[column][1])))
    
    df_column = pd.DataFrame(columns=[time, column])
    df_column[time] = result[column][0]
    df_column[column] = result[column][1]
    df_column.set_index(time, inplace = True)

    df = df.join(df_column, how='outer')

For column IP.NSRCGEN:BIASDISCAQNV 8461 datapoints exist.
For column IP.NSRCGEN:GASSASAQN 8525 datapoints exist.
For column IP.NSRCGEN:SOURCEHTAQNI 249022 datapoints exist.
For column IP.SOLCEN.ACQUISITION:CURRENT 1567 datapoints exist.
For column IP.SOLEXT.ACQUISITION:CURRENT 16906 datapoints exist.
For column IP.NSRCGEN:OVEN1AQNP 8555 datapoints exist.
For column IP.NSRCGEN:OVEN2AQNP 8860 datapoints exist.
For column ITF.BCT25:CURRENT 261345 datapoints exist.


In [7]:
filename = '../Data_Raw/Nov2015.csv'

In [8]:
df.to_csv(filename)